### Some functions 

In [21]:
# Make prediction
def make_prediction(model_name, model1, model2, X_final_1, X_final_2, final_val_dat):
    pred_1 = pd.DataFrame(model1.predict(X_final_1.iloc[:,1:8]))
    id_1 = pd.DataFrame(X_final_1['id']).reset_index(drop=True)
    df_new = pd.concat([id_1, pred_1], axis=1)
    pred_2 = pd.DataFrame(model2.predict(X_final_2.iloc[:,1:7]))
    id_2 = pd.DataFrame(X_final_2['id']).reset_index(drop=True)
    df2 = id_2.join(pred_2)
    df_new = df_new.append(df2) 
    df_new = df_new.set_index('id')
    df_new = df_new.reindex(index=final_val_dat['id'])
    df_new = df_new.reset_index() 
    df_new = df_new.rename(columns={0: model_name})
    return df_new

# Make predictions for certain models: like XGB and Voting Classifier which
# do not accept dataframes 
def make_prediction2(model_name, model1, model2, X_final_1, X_final_2, final_val_dat):
    X_final_col1 = X_final_1.drop('id', axis=1).values
    X_final_col2 = X_final_2.drop('id', axis=1).values
    pred_1 = pd.DataFrame(model1.predict(X_final_col1))
    id_1 = pd.DataFrame(X_final_1['id']).reset_index(drop=True)
    df_new = pd.concat([id_1, pred_1], axis=1)
    pred_2 = pd.DataFrame(model2.predict(X_final_col2))
    id_2 = pd.DataFrame(X_final_2['id']).reset_index(drop=True)
    df2 = id_2.join(pred_2)
    df_new = df_new.append(df2) 
    df_new = df_new.set_index('id')
    df_new = df_new.reindex(index=final_val_dat['id'])
    df_new = df_new.reset_index()
    df_new = df_new.rename(columns={0: model_name})
    return df_new

# Function for plotting ROC and Precision Recall curve
def plot_roc_and_pr_curves (model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:,1] #prob for positive outcome only
    model_precision, model_recall, _ = precision_recall_curve(y_test, y_pred_prob)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    fig, ax = plt.subplots(1, 2, figsize=(12, 3))  
    ax[0].plot([0, 1], [0, 1], 'k--')
    ax[0].plot(fpr, tpr)   #ROC curve
    ax[0].set_xlabel('False Positive Rate')
    ax[0].set_ylabel('True Positive Rate')
    ax[0].set_title('ROC Curve') 
    ax[1].plot(model_recall, model_precision, label='Logistic') #PR curve
    ax[1].set_xlabel('Recall')
    ax[1].set_ylabel('Precision')
    ax[1].set_title('Precision Recall Curve')


# Function for printing metrics
def print_model_metrics(model, X_test, y_test, y_pred, model_nm, sum_table, tuning=False, roc_pr=False):
    acc = metrics.accuracy_score(y_test, y_pred)
    av_pr = metrics.average_precision_score(y_test, y_pred)
    auc = metrics.roc_auc_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print("Accuracy: ", acc)
    print("AUC: ", auc)
    print("Average Precision: ", av_pr)
    print("f1: ", f1)
    print(classification_report(y_test, y_pred, digits=4))
    if roc_pr is True: 
        plot_roc_and_pr_curves (model, X_test, y_test)
    if tuning is True: 
        best_param = model.best_params_
        best_acc = model.best_score_
        print("Tuned Parameters: ", best_param)
        print("Best model accuracy: ", best_acc)
        sum_table  = sum_table.append({'model': model_nm, 'acc': acc, 'f1':f1, 'auc':auc,
                                       'av_pr':av_pr,'best_acc':best_acc},ignore_index=True)
    else: 
        sum_table  = sum_table.append({'model': model_nm, 'acc': acc, 'f1':f1, 'auc':auc,
                                       'av_pr':av_pr,'best_acc': acc},ignore_index=True)
    return sum_table 